# Multilayer perceptrons (MLP)

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

### Univariate MLP models

#### Data preparation

In [6]:
# choose a number of time steps
n_steps = 3
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [52]:
# Split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # Find the end of this pattern
        end_ix = i + n_steps
        # Check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i: end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [7]:
# split into samples
X, y = split_sequence(raw_seq, n_steps)

#### Define model

In [10]:
univariate_model = Sequential()
univariate_model.add(Dense(100, activation='relu', input_dim=n_steps))
univariate_model.add(Dense(1))
univariate_model.compile(optimizer='adam', loss='mse')

#### Fit model

In [21]:
univariate_model.fit(X, y, epochs = 500, verbose = 0)

#### Prediction

In [23]:
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps))
yhat = univariate_model.predict(x_input, verbose = 1)
print(yhat)

1/1 [==============================] - 0s 21ms/step
[[101.919846]]


### Multivariate MLP models

#### Data preparation

In [30]:
# choose a number of time steps
n_steps = 3

# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
print("dataset: ", out_seq)

dataset:  [ 25  45  65  85 105 125 145 165 185]


In [29]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print("reshaped dataset: \n", dataset)

reshaped dataset: 
 [[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [36]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [60]:
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print("X shape, y shape: ", X.shape, y.shape)
# summarize the data
print("data summary: ")
for i in range(len(X)):
    print(X[i], y[i])

X shape, y shape:  (7, 3, 2) (7,)
data summary: 
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [61]:
n_input = X.shape[1]*X.shape[2]
X = X.reshape((X.shape[0], n_input))

In [63]:
print(X)

[[10 15 20 25 30 35]
 [20 25 30 35 40 45]
 [30 35 40 45 50 55]
 [40 45 50 55 60 65]
 [50 55 60 65 70 75]
 [60 65 70 75 80 85]
 [70 75 80 85 90 95]]


#### Define model

In [45]:
multivariate_model = Sequential()
multivariate_model.add(Dense(100, activation='relu', input_dim=n_input))
multivariate_model.add(Dense(1))
multivariate_model.compile(optimizer='adam', loss='mse')

#### Fit model

In [46]:
multivariate_model.fit(X, y, epochs = 200, verbose = 0)

#### Prediction

In [49]:
# demonstrate prediction
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_input))
yhat = multivariate_model.predict(x_input, verbose=0)
print(yhat)

[[213.26671]]


### Multi-headed MLP models

#### Data preparation

In [53]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import concatenate

In [55]:
# choose a number of time steps
n_steps = 3

# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
print("dataset: ", out_seq)

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print("reshaped dataset: \n", dataset)

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print("X shape, y shape: ", X.shape, y.shape)
# summarize the data
print("data summary: ")
for i in range(len(X)):
    print(X[i], y[i])



dataset:  [ 25  45  65  85 105 125 145 165 185]
reshaped dataset: 
 [[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
X shape, y shape:  (7, 3, 2) (7,)
data summary: 
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [58]:
# separate input data
X1 = X[:, :, 0]
X2 = X[:, :, 1]
print("X1: \n", X1)
print("X2: \n", X2)

X1: 
 [[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]
 [70 80 90]]
X2: 
 [[15 25 35]
 [25 35 45]
 [35 45 55]
 [45 55 65]
 [55 65 75]
 [65 75 85]
 [75 85 95]]


#### Define model

In [64]:
# first input model
visible1 = Input(shape=(n_steps,))
dense1 = Dense(100, activation='relu')(visible1)
# second input model
visible2 = Input(shape=(n_steps,))
dense2 = Dense(100, activation='relu')(visible2)

#### Merge model

In [65]:
# merge input models
merge = concatenate([dense1, dense2])
output = Dense(1)(merge)
model = Model(inputs=[visible1, visible2], outputs=output)
model.compile(optimizer='adam', loss='mse')

#### Fit model

In [66]:
# fit model
model.fit([X1, X2], y, epochs=2000, verbose=0)

In [67]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps))
x2 = x_input[:, 1].reshape((1, n_steps))
yhat = model.predict([x1, x2], verbose=0)
print(yhat)

[[205.96945]]


### Multi-step MLP models

### Multivariate multi-step MLP models